In [1]:
import pandas as pd
import os

In [2]:
repseq_df = pd.read_csv("repseq.microcosms.tmp.tsv", sep = "\t")

In [3]:
repseq_df

,Query,EC,confidence,domain
0,2P0U_A,2.3.1.156,0.9830,euk72_ep
1,5YPT_F,2.3.1.156,0.8470,euk72_ep
2,AAB22421.1,3.1.26.5,0.5042,euk72_ep
3,AAF05807.1,7.1.2.2,0.9620,euk72_ep
4,AAL73350.1,1.2.1.12,0.9644,euk72_ep
...,...,...,...,...
6171038,YP_654087.1,7.1.1.2,0.9779,euk72_ep
6171039,YP_654221.1,4.1.1.39,0.9989,euk72_ep
6171040,YP_740422.1,7.1.1.2,0.8803,euk72_ep
6171041,YP_874516.1,3.6.4.12,0.7892,euk72_ep


In [4]:
accessions_df = repseq_df.loc[:,["Query", "EC", "domain"]]
accessions_df["ISEUK"] = accessions_df["domain"].str.contains("euk")

In [60]:
ec_bothdom = accessions_df.groupby("EC")["ISEUK"].apply(lambda x:len(x.unique()) == 2)
ec_bothdom_list = list(ec_bothdom[ec_bothdom].index)

In [64]:
accessions_df_rel = accessions_df[accessions_df["EC"].isin(ec_bothdom_list)]

In [67]:
accessions_df_rel

,Query,EC,domain,ISEUK
2,AAB22421.1,3.1.26.5,euk72_ep,True
3,AAF05807.1,7.1.2.2,euk72_ep,True
4,AAL73350.1,1.2.1.12,euk72_ep,True
5,AAN74743.1,2.6.1.27,euk72_ep,True
6,AAO46122.1,3.6.4.10,euk72_ep,True
...,...,...,...,...
6171038,YP_654087.1,7.1.1.2,euk72_ep,True
6171039,YP_654221.1,4.1.1.39,euk72_ep,True
6171040,YP_740422.1,7.1.1.2,euk72_ep,True
6171041,YP_874516.1,3.6.4.12,euk72_ep,True


In [70]:
acc_counts = accessions_df_rel.groupby(["EC", "ISEUK"]).count().reset_index()
ecs_enough_eukmembers = set(acc_counts[(acc_counts["ISEUK"] == True) & (acc_counts["Query"] > 5)].EC)

In [79]:
accessions_outdf

,EC,ISEUK,Query
0,1.1.1.1,False,AEJ53306_1\nAKJ28696_1\nAKR51806_1\nAKT34022_1...
1,1.1.1.1,True,CEM14143.1\nEP00001P000163\nEP00001P117662\nEP...
2,1.1.1.10,False,AHM62958_1\nALS79891_1\nANU10707_1\nAPD05872_1...
3,1.1.1.10,True,EP00005P005247\nEP00008P005642\nEP00008P010001...
4,1.1.1.100,False,ABY71998_1\nAFS48457_1\nAFS49619_1\nAGY53885_1...
...,...,...,...
6803,7.6.2.5,True,EP00163P007010\nEP00163P013418\nEP00270P001738...
6804,7.6.2.7,False,AFB78298_1\nAGB78726_1\nAHF89345_1\nAHF90646_1...
6805,7.6.2.7,True,EP00163P012848\nEP00163P017165\nEP00873P030931...
6806,7.6.2.8,False,ACX82737_1\nADB37613_1\nAGB77936_1\nAGB79451_1...


In [78]:
accessions_outdf = accessions_df_rel.groupby(["EC", "ISEUK"])["Query"].apply(lambda x: "\n".join(x)).reset_index()
accessions_outdf = accessions_outdf[accessions_outdf["EC"].isin(ecs_enough_eukmembers)]

In [80]:
outpathroot = "/data/luojaa/eukgen/processing/microcosm_dl/"

In [81]:
accessions_outdf

,EC,ISEUK,Query
0,1.1.1.1,False,AEJ53306_1\nAKJ28696_1\nAKR51806_1\nAKT34022_1...
1,1.1.1.1,True,CEM14143.1\nEP00001P000163\nEP00001P117662\nEP...
2,1.1.1.10,False,AHM62958_1\nALS79891_1\nANU10707_1\nAPD05872_1...
3,1.1.1.10,True,EP00005P005247\nEP00008P005642\nEP00008P010001...
4,1.1.1.100,False,ABY71998_1\nAFS48457_1\nAFS49619_1\nAGY53885_1...
...,...,...,...
6803,7.6.2.5,True,EP00163P007010\nEP00163P013418\nEP00270P001738...
6804,7.6.2.7,False,AFB78298_1\nAGB78726_1\nAHF89345_1\nAHF90646_1...
6805,7.6.2.7,True,EP00163P012848\nEP00163P017165\nEP00873P030931...
6806,7.6.2.8,False,ACX82737_1\nADB37613_1\nAGB77936_1\nAGB79451_1...


In [82]:
ecs = list(accessions_outdf["EC"])
iseuks = list(accessions_outdf["ISEUK"])
uids = list(accessions_outdf["Query"])

In [12]:
# with open("/data/luojaa/eukgen/mmseqs/ran/bothdomains.ids", "w") as outfile:
#     for i in range(len(accessions_outdf)):
#         uidchunk = uids[i]
#         for uid in uidchunk.split("\n"):
#             print(uid,file=outfile)

In [ ]:
# for i in range(len(accessions_outdf)):
#     ec, iseuk, uid = ecs[i], iseuks[i], uids[i]
#     os.system(f"mkdir {outpathroot}{ec}")
#     if iseuk:
#         outpath = f"{outpathroot}{ec}/{ec}.query.acc"
#         with open(outpath, "w") as outfile:
#             print(uid, file=outfile)
#     else:
#         outpath = f"{outpathroot}{ec}/{ec}.target.acc"
#         with open(outpath, "w") as outfile:
#             print(uid, file=outfile)